# Computer Vision Project

#### Approach we will take for the Aerial vs Ground Natural Disaster computer vision project

## Import Packages

In [3]:
# Get this from GitHub Repository / Random Class Notebooks / etc.

# At minimum, we need PyTorch

## Load data to our project

In [4]:
# Reference the following notebooks:

# Identifying Pneumothorax in chest xrays
# Image classification (writing neural network from scratch)

## Data Preparation: 
### Set up class to add labels, manipulate the images to size correctly, data augmentation

In [5]:
# AIDER - use approach based on image classification notebook (writing neural network from scratch)
# MEDIC - need to use DICOM

# labels = Aerial and Ground. Need to standardize "Fire", "Flood", "No Disaster" across both datasets so labels are the same

# Reference code to size images correctly. May not be in correct section, 
# but we might need to compare image sizes across our two data sets and standardize?

# Data augmentation to create new images from AIDER data set to get better class balance between aerial and ground

## Data Manipulation

In [6]:
#MEDIC:
## Choose which pictures to keep from MEDIC (there are a lot more than AIDER)
## Drop pictures we don't want from MEDIC
## Drop Mild Category
## Drop Not Informative label if they exist?
## Only Keep fires, floods, and not disasters

#AIDER:
## Only keep fires, floods and not disasters


## Combine Datasets

In [7]:
# Merge MEDIC and AIDER

## Train_test_split into our training and test set

In [8]:
# Set up training, validation, and test sets

# Amani:
## Set up Model Architecture

In [17]:
# Need to decide on best architecture to use based on our goals. Performance/computational resources tradeoffs

# Are there pre-trained models that are less computationally heavy to start off with? Research this & include in PowerPoint
# Transfer Learning ^

## Train model 

In [15]:
# Reference existing code to train model

## Evaluate Performance

In [16]:
# Validation / Test set

## Report on Statistics

In [11]:
# Performance, recall, F1-score

In [12]:
# Ground vs Aerial analysis